In [22]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import (
    ImageDataGenerator, 
    DirectoryIterator
)
from keras.applications.resnet50 import ResNet50, preprocess_input

import pathlib

import sys

sys.path.insert(0, "ml_project_4_face_detection/backend")
import recognition
import json

In [9]:
path_to_saved_model = pathlib.Path("ml_project_4_face_detection/backend/model")
assert path_to_saved_model.exists(), f"Could not find {path_to_saved_model}"
assert path_to_saved_model.is_dir(), f"{path_to_saved_model} is not a directory"

train=False
if not train:
    model = recognition.load_local_model(path_to_saved_model)

2023-07-25 04:35:01.770439: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2023-07-25 04:35:01.770459: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 96.00 GB
2023-07-25 04:35:01.770465: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 36.00 GB
2023-07-25 04:35:01.770499: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-25 04:35:01.770517: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


First, download the pre-trained ResNet 50 model from TensorFlow Hub

In [13]:
base_model = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3),
)

Then, figure out how many celebrities we have (just the number of directories inside the `Celebrity Faces Dataset`)

In [11]:
path_to_celebrity_dataset = pathlib.Path("ml_project_4_face_detection/backend/Celebrity Faces Dataset/")
assert path_to_celebrity_dataset.exists(), f"Could not find {path_to_celebrity_dataset}."
number_celebrities = len(list(path_to_celebrity_dataset.iterdir()))
print(f"There are {number_celebrities} celebrities in the dataset.")

There are 17 celebrities in the dataset.


Now, we'll instantiate a model on top of our pre-trained ResNet 50 model.

In [14]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(number_celebrities, activation="softmax"),
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 17)                34833     
                                                                 
Total params: 23622545 (90.11 MB)
Trainable params: 23569425 (89.91 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


Now we'll use TensorFlow utilities (`ImageDataGenerator`) to load and preprocess the celebrity images

In [15]:
target_image_size = recognition.DEFAULT_IMAGE_SIZE

training_data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

training_data_streamer: DirectoryIterator = training_data_generator.flow_from_directory(
    path_to_celebrity_dataset,
    target_size=target_image_size,
    batch_size=32,
    class_mode="categorical",
)


Found 1800 images belonging to 17 classes.


Now we'll compile our model, using the Adam optimizer, and the categorical crossentropy loss function

In [16]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [17]:
training_epochs = 10

history = model.fit(
    training_data_streamer,
    epochs=training_epochs,
)

Epoch 1/10


2023-07-25 04:37:31.147941: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


57/57 [==============================] - 15s 203ms/step - loss: 1.6209 - accuracy: 0.4961
Epoch 2/10
57/57 [==============================] - 11s 182ms/step - loss: 0.3941 - accuracy: 0.8961
Epoch 3/10
57/57 [==============================] - 10s 181ms/step - loss: 0.1561 - accuracy: 0.9667
Epoch 4/10
57/57 [==============================] - 11s 195ms/step - loss: 0.0864 - accuracy: 0.9850
Epoch 5/10
57/57 [==============================] - 11s 196ms/step - loss: 0.0754 - accuracy: 0.9822
Epoch 6/10
57/57 [==============================] - 11s 198ms/step - loss: 0.0403 - accuracy: 0.9933
Epoch 7/10
57/57 [==============================] - 11s 191ms/step - loss: 0.0466 - accuracy: 0.9883
Epoch 8/10
57/57 [==============================] - 11s 191ms/step - loss: 0.0538 - accuracy: 0.9833
Epoch 9/10
57/57 [==============================] - 11s 198ms/step - loss: 0.0718 - accuracy: 0.9817
Epoch 10/10
57/57 [==============================] - 12s 201ms/step - loss: 0.0431 - accuracy: 0.9883


Now, we use this fine-tuned model to make predictions:

In [20]:
example_image = recognition.load_image_from_file(
    image_path=path_to_celebrity_dataset / "Will Smith/001_beebcee2.jpg"
)

idx_to_celebrity_name = {v: k for k, v in training_data_streamer.class_indices.items()}

predicted_celebrity = recognition.make_prediction(example_image, model, idx_to_celebrity_name)
print(f"Predicted celebrity: {predicted_celebrity}")

2023-07-25 04:40:26.567335: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 773ms/step


2023-07-25 04:40:27.096 | INFO     | recognition:make_prediction:70 - Predicted celebrity index: 16


Predicted celebrity: Will Smith


Save our fine-tuned ResNet 50 to a `model` directory in the `backend` folder in the SavedModel format, making sure to attach the index to celebrity name dictionary as an asset inside the model.

In [33]:
path_to_saved_model.parent

PosixPath('ml_project_4_face_detection/backend')

In [34]:
with open(path_to_saved_model.parent / "idx_to_celebrity_name.json", 'w') as f:
    json.dump(idx_to_celebrity_name, f)
asset = tf.saved_model.Asset('idx_to_celebrity_name.json')

model.asset = asset
model.save(path_to_saved_model)

INFO:tensorflow:Assets written to: ml_project_4_face_detection/backend/model/assets


INFO:tensorflow:Assets written to: ml_project_4_face_detection/backend/model/assets


And here is an example of how you'd load the model back in, along with its dictionary of labels:

In [35]:
loaded_model = tf.keras.models.load_model(path_to_saved_model)
with open(loaded_model.asset.asset_path.numpy(), 'r') as f:
    loaded_idx_to_celebrity_name= json.load(f)

print(loaded_idx_to_celebrity_name)

{'0': 'Angelina Jolie', '1': 'Brad Pitt', '2': 'Denzel Washington', '3': 'Hugh Jackman', '4': 'Jennifer Lawrence', '5': 'Johnny Depp', '6': 'Kate Winslet', '7': 'Leonardo DiCaprio', '8': 'Megan Fox', '9': 'Natalie Portman', '10': 'Nicole Kidman', '11': 'Robert Downey Jr', '12': 'Sandra Bullock', '13': 'Scarlett Johansson', '14': 'Tom Cruise', '15': 'Tom Hanks', '16': 'Will Smith'}
